In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import xarray as xr
from bibliotheque import *
from params import *

## PARAMS

In [3]:
srate = down_srate
srate

250

In [4]:
save = True

In [5]:
ecg_interesting_metrics = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF']
ppg_interesting_metrics = ['mean amplitude', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20']
eda_interesting_metrics = ['nb_peaks', 'mean_height', 'EDA_Symp', 'EDA_SympN']

## TOOLS

In [6]:
def eeg_to_metrics(eeg_sig, srate=srate):
    metrics = nk.eeg_power(eeg_sig, sampling_rate=srate)
    return metrics
    
def ecg_to_metrics(ecg_sig, interesting_metrics, srate=srate, get_fci_signal=False):
    peaks, info_ecg = nk.ecg_peaks(ecg_sig, sampling_rate=srate,method='neurokit', correct_artifacts=True)
    R_peaks = info_ecg['ECG_R_Peaks'] # get R time points
    metrics = pd.concat([nk.hrv_time(R_peaks, sampling_rate=srate), nk.hrv_frequency(R_peaks, sampling_rate=srate)], axis = 1)
    
    if get_fci_signal:
        diff_R_peaks = np.diff(R_peaks) 
        x = time_vector(ecg, srate)
        xp = R_peaks[1::]/srate
        fp = diff_R_peaks
        interpolated_hrv = np.interp(x, xp, fp, left=None, right=None, period=None) / srate
        fci = 60 / interpolated_hrv
        
        return metrics[interesting_metrics], fci
    else:
        return metrics[interesting_metrics]
    
def gsr_homemade_metrics(info, dict_symp):   
    nb_peaks = info['SCR_Peaks'].size
    mean_height = np.mean(info['SCR_Height'])
    mean_amp = np.mean(info['SCR_Amplitude'])
    mean_risetime = np.mean(info['SCR_RiseTime'])
    mean_recov_time = np.mean(info['SCR_RecoveryTime'])
    
    data = [nb_peaks, mean_height, mean_amp, mean_risetime, mean_recov_time, dict_symp['EDA_Symp'], dict_symp['EDA_SympN']]
    df = pd.Series(data=data, index = ['nb_peaks', 'mean_height', 'mean_amp', 'mean_risetime', 'mean_recov_time','EDA_Symp','EDA_SympN'])
    return df

def eda_to_metrics(eda_sig, interesting_metrics, srate=srate):
    __ , info = nk.eda_process(eda_sig, sampling_rate=srate, method='neurokit')
    dict_symp = nk.eda_sympathetic(eda_sig, sampling_rate = srate)
    metrics = gsr_homemade_metrics(info, dict_symp)
    return metrics[interesting_metrics]
    
def ppg_amplitude(sig, srate, show=False):
    idx_maxs = signal.find_peaks(sig, distance = srate/2)[0]
    idx_mins = signal.find_peaks(-sig, distance = None)[0]
 
    amplitudes = []
    val_creux = []
    val_sommets = []
    indices_creux = []
    indices_sommets = []
    for idx_sommet in idx_maxs:
        cond = idx_mins < idx_sommet
        if not sum(cond) == 0:
            idx_creux = idx_mins[cond][-1]
            value_creux = sig[idx_creux]
            value_sommet = sig[idx_sommet]
            
            
            val_creux.append(value_creux)
            val_sommets.append(value_sommet)
            indices_sommets.append(idx_sommet)
            indices_creux.append(idx_creux)
        
            amplitude = value_sommet - value_creux
            amplitudes.append(amplitude)
    # print(np.mean(amplitudes))
    

    # df = pd.DataFrame(np.array([np.arange(1,len(val_sommets)+1 , 1), indices_sommets , indices_creux, val_sommets, val_creux, amplitudes]).T, columns = ['cycle','idx_max','idx_min','val_max','val_min','amplitude'])

    if show:
        plt.figure()
        plt.plot(sig)
        plt.plot(indices_sommets, val_sommets, 'x')
        plt.plot(indices_creux, val_creux, 'o')
        plt.show()
        
    return np.mean(amplitudes)

def ppg_to_metrics(ppg_sig, interesting_metrics, srate=srate): 
    peaks = nk.ppg_findpeaks(ppg_sig, sampling_rate=srate, method='elgendi', show=False)
    metrics = pd.concat([nk.hrv_time(peaks, sampling_rate=srate), nk.hrv_frequency(peaks, sampling_rate=srate)], axis = 1)
    amp = ppg_amplitude(ppg_sig, srate)
    metrics.insert(0, 'mean amplitude', amp)
    return metrics[interesting_metrics]

## LOAD DATA

In [7]:
da = xr.load_dataarray('../data_preprocessed/da_cleaned.nc').sel(cleaning = 'clean') # sel already cleaned data

In [8]:
da

<xarray.DataArray (participant: 19, room: 9, biosig: 4, time: 22501)>
array([[[[-1.70530257e-13,  2.07324372e+01,  6.67476693e+00, ...,
           1.75077331e+01, -4.43587850e+01, -5.68434189e-14],
         [-3.99481870e+00, -3.23349538e+00, -2.40230460e+00, ...,
           9.78090929e+00,  5.59578507e+00,  3.78464397e+00],
         [ 1.35440469e+02,  1.35449113e+02,  1.35457694e+02, ...,
           8.01139701e+01,  8.01139069e+01,  8.01138556e+01],
         [ 2.75827751e+01,  2.98107144e+01,  3.18742659e+01, ...,
          -1.12543724e+01, -1.29490419e+01, -1.47835627e+01]],

        [[ 7.10542736e-14, -1.90357944e+01,  4.12414650e+01, ...,
          -2.12264194e+02, -2.43733258e+02, -1.20792265e-13],
         [ 9.72328585e-01,  2.62541341e+00,  4.91714400e+00, ...,
           4.97008135e+00,  6.03783369e+00,  7.11076450e+00],
         [ 1.07809219e+02,  1.07891931e+02,  1.07973620e+02, ...,
           1.40340527e+02,  1.40341398e+02,  1.40342099e+02],
         [-2.69893796e-02,  2.01497261e+00,  3.89058492e+00, ...,
           2.18422499e+01,  2.15076903e+01,  2.09119067e+01]],

        [[ 4.26325641e-14,  1.30152173e+02,  2.60755517e+02, ...,
          -7.80999512e+01, -3.82789252e+01,  9.23705556e-14],
...
         [ 1.44611218e+01,  1.64178302e+01,  1.81252815e+01, ...,
           1.45033800e+00,  3.28447173e-02, -1.49519202e+00]],

        [[-8.52651283e-14, -1.80235105e+00, -9.58826987e+01, ...,
           1.45491944e+02,  1.94329524e+02, -1.24344979e-13],
         [-1.37971192e+00, -1.01666121e+00, -8.17942846e-01, ...,
          -4.68130223e+01, -3.12183358e+01, -1.47638938e+01],
         [ 1.51222263e+02,  1.51230348e+02,  1.51238320e+02, ...,
           1.33332956e+02,  1.33332920e+02,  1.33332893e+02],
         [ 3.91119195e+00,  6.29535336e+00,  8.50622546e+00, ...,
          -1.64113573e+01, -1.79739305e+01, -1.96548977e+01]],

        [[ 2.13162821e-13,  1.13195147e+02,  1.13290008e+02, ...,
           2.52931199e+00, -1.76723928e+01,  9.94759830e-14],
         [ 4.02698987e+00,  4.61290739e+00,  5.14984887e+00, ...,
           9.92205147e+00,  1.17204462e+01,  1.35628190e+01],
         [ 1.14702888e+02,  1.14728799e+02,  1.14754523e+02, ...,
           1.19601624e+02,  1.19601607e+02,  1.19601594e+02],
         [-6.26258120e+00, -3.50812536e+00, -9.25309278e-01, ...,
           3.03212115e+00,  1.15212188e+00, -8.81779910e-01]]]])
Coordinates:
  * participant  (participant) object 'P01PPILNI' 'P05PROZEM' ... 'P30BATDI'
  * room         (room) object 'Baseline' 'Take-off' ... 'End of the world'
  * biosig       (biosig) object 'EEG' 'ECG' 'EDA' 'PPG'
    cleaning     <U5 'clean'
  * time         (time) float64 0.0 0.004 0.008 0.012 ... 89.99 89.99 90.0 90.0

## DATA TO METRICS

In [9]:
df_quality = pd.read_excel('../data_preprocessed/quality_encoding.xlsx', index_col = 0).set_index(['biosig','participant'])

In [10]:
df_quality

quality
biosig participant         
EEG    P01PPILNI          0
       P03PBABCO          0
       P04PROMCH          0
       P05PROZEM          2
       P06PHERAX          0
...                     ...
PPG    P26BOUHE           0
       P27OSTMA           0
       P28JUDGU           0
       P29FICMA           0
       P30BATDI           0

[116 rows x 1 columns]

In [11]:
eeg_concat = []
ecg_concat = []
eda_concat = []
ppg_concat = []

uncomputed_participants = {}

for participant in participants:
    print(participant)
    for room in rooms:
        # print(room)
        for biosig in biosigs:

            sig = da.loc[participant, room, biosig, :].values        
            quality = df_quality.loc[(biosig,participant),'quality']
                
            # print(participant , biosig, quality)
            if not np.sum(sig) == 0 : # don't compute metrics if no signal

                if biosig == 'EEG':
                    metrics = eeg_to_metrics(eeg_sig=sig)
                    metrics.insert(0, 'chan', biosig)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'quality', quality)
                    metrics.insert(0, 'participant', participant)
                    eeg_concat.append(metrics)
                elif biosig == 'ECG':
                    metrics = ecg_to_metrics(ecg_sig=sig, interesting_metrics = ecg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'quality', quality)
                    metrics.insert(0, 'participant', participant)
                    ecg_concat.append(metrics)
                elif biosig == 'EDA':
                    metrics = eda_to_metrics(eda_sig=sig, interesting_metrics = eda_interesting_metrics)
                    metrics['room'] = room
                    metrics['quality'] =  quality
                    metrics['participant'] = participant
                    eda_concat.append(metrics)
                elif biosig == 'PPG':
                    metrics = ppg_to_metrics(ppg_sig=sig, interesting_metrics = ppg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'quality', quality)
                    metrics.insert(0, 'participant', participant)
                    ppg_concat.append(metrics)     

        else:
            uncomputed_participants[participant] = biosig
                
eeg_metrics = pd.concat(eeg_concat).drop(columns = ['Channel'])
ecg_metrics = pd.concat(ecg_concat)
eda_metrics = pd.concat(eda_concat, axis = 1).T
ppg_metrics = pd.concat(ppg_concat)

print(uncomputed_participants)

P01PPILNI


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P05PROZEM


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P07GHOLE


...

/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

P30BATDI


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {

{'P01PPILNI': 'PPG', 'P05PROZEM': 'PPG', 'P07GHOLE': 'PPG', 'P08AKKOR': 'PPG', 'P10LEVVA': 'PPG', 'P11KERSA': 'PPG', 'P12BOULI': 'PPG', 'P14BENLA': 'PPG', 'P15LEPMA': 'PPG', 'P16MAUAD': 'PPG', 'P17ETRPA': 'PPG', 'P19MONAL': 'PPG', 'P20POUAX': 'PPG', 'P21LIYAT': 'PPG', 'P22DALPI': 'PPG', 'P25PEIAN': 'PPG', 'P27OSTMA': 'PPG', 'P28JUDGU': 'PPG', 'P30BATDI': 'PPG'}


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '


In [12]:
da_visit = xr.load_dataarray('../data_preprocessed/da_visit_effect_cleaned.nc').sel(cleaning = 'clean') # sel already cleaned data

In [13]:
eeg_concat = []
ecg_concat = []
eda_concat = []

for participant in participants:
    print(participant)
    for epoch in da_visit.coords['epoch'].values:
        # print(room)
        for biosig in biosigs:
            # print(biosig)
            sig = da_visit.loc[participant, epoch, biosig, :].values
            quality = df_quality.loc[(biosig,participant),'quality']
                
            if biosig == 'EEG':
                metrics = eeg_to_metrics(eeg_sig=sig)
                metrics.insert(0, 'chan', biosig)
                metrics.insert(0, 'epoch', epoch)
                metrics.insert(0, 'quality', quality)
                metrics.insert(0, 'participant', participant)
                eeg_concat.append(metrics)
            elif biosig == 'ECG':
                metrics = ecg_to_metrics(ecg_sig=sig, interesting_metrics = ecg_interesting_metrics)
                metrics.insert(0, 'epoch', epoch)
                metrics.insert(0, 'quality', quality)
                metrics.insert(0, 'participant', participant)
                ecg_concat.append(metrics)
            elif biosig == 'EDA':
                metrics = eda_to_metrics(eda_sig=sig, interesting_metrics = eda_interesting_metrics)
                metrics['epoch'] = epoch
                metrics['quality'] =  quality
                metrics['participant'] = participant
                eda_concat.append(metrics) 
                
eeg_metrics_visit = pd.concat(eeg_concat).drop(columns = ['Channel'])
ecg_metrics_visit = pd.concat(ecg_concat)
eda_metrics_visit = pd.concat(eda_concat, axis = 1).T

P01PPILNI
P05PROZEM
P07GHOLE
P08AKKOR
P10LEVVA
P11KERSA
P12BOULI
P14BENLA
P15LEPMA
P16MAUAD
P17ETRPA


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/neurokit2/signal/signal_fixpeaks.py:277: RuntimeWarning: divide by zero encountered in true_divide
  mrrs /= th2
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/neurokit2/signal/signal_fixpeaks.py:277: RuntimeWarning: invalid value encountered in true_divide
  mrrs /= th2


P19MONAL


/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


P20POUAX
P21LIYAT
P22DALPI
P25PEIAN
P27OSTMA
P28JUDGU
P30BATDI


In [14]:
if save:
    eeg_metrics.to_excel('../metrics/eeg_metrics.xlsx')
    ecg_metrics.to_excel('../metrics/ecg_metrics.xlsx')
    eda_metrics.to_excel('../metrics/eda_metrics.xlsx')
    ppg_metrics.to_excel('../metrics/ppg_metrics.xlsx')
    
    eeg_metrics_visit.to_excel('../metrics/eeg_metrics_visit.xlsx')
    ecg_metrics_visit.to_excel('../metrics/ecg_metrics_visit.xlsx')
    eda_metrics_visit.to_excel('../metrics/eda_metrics_visit.xlsx')